# March Madness Predictions

Jason Lott, Brendan LaPuma, Eamon Weingold

In [1]:
import pandas as pd, numpy as np
import tensorflow as tf
import re
from itertools import product

2023-03-12 22:03:58.499770: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-12 22:03:58.615836: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-12 22:03:58.620255: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-12 22:03:58.620271: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
df = pd.read_csv('data/MRegularSeasonDetailedResults.csv')

In [3]:
winners = df.iloc[:, np.r_[0:4, 6:21]]
losers = df.iloc[:, np.r_[0, 1, 4:8, 21:34]]

In [4]:
winners

,Season,DayNum,WTeamID,WScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF
0,2003,10,1104,68,N,0,27,58,3,14,11,18,14,24,13,23,7,1,22
1,2003,10,1272,70,N,0,26,62,8,20,10,19,15,28,16,13,4,4,18
2,2003,11,1266,73,N,0,24,58,8,18,17,29,17,26,15,10,5,2,25
3,2003,11,1296,56,N,0,18,38,3,9,17,31,6,19,11,12,14,2,18
4,2003,11,1400,77,N,0,30,61,6,14,11,13,17,22,12,14,4,4,20
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106829,2023,113,1403,74,A,0,29,47,7,12,9,14,5,31,11,16,4,1,17
106830,2023,113,1405,84,H,0,32,61,10,20,10,13,7,25,16,7,7,7,17
106831,2023,113,1429,65,A,0,21,51,7,27,16,22,7,27,11,12,7,3,15
106832,2023,113,1433,88,A,0,31,59,11,19,15,18,8,23,14,9,6,5,16


In [5]:
losers

,Season,DayNum,LTeamID,LScore,WLoc,NumOT,LFGM,LFGA,LFGM3,LFGA3,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF
0,2003,10,1328,62,N,0,22,53,2,10,16,22,10,22,8,18,9,2,20
1,2003,10,1393,63,N,0,24,67,6,24,9,20,20,25,7,12,8,6,16
2,2003,11,1437,61,N,0,22,73,3,26,14,23,31,22,9,12,2,5,23
3,2003,11,1457,50,N,0,18,49,6,22,8,15,17,20,9,19,4,3,23
4,2003,11,1208,71,N,0,24,62,6,16,17,27,21,15,12,10,7,1,14
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106829,2023,113,1328,63,A,0,21,59,9,33,12,15,6,13,13,8,10,1,12
106830,2023,113,1103,63,H,0,26,59,4,21,7,14,9,23,10,10,3,0,11
106831,2023,113,1461,55,A,0,17,54,6,23,15,20,7,25,6,11,6,3,19
106832,2023,113,1386,63,A,0,21,56,8,28,13,18,5,20,10,11,3,3,15


Adding a "Result" column. If GREATER id wins, 1. If LOWER id wins, 0. 

Also, re-organizing the table from winner's stats and loser's stats to higher/lower team ID's stats. 

In [6]:
df2 = df.copy(deep= True)
df2["Result"] = np.select([df["WTeamID"] > df["LTeamID"], df["WTeamID"] < df["LTeamID"]], [1, 0])
idx = (df2["Result"] == 0)
df2.loc[idx,["WTeamID", "WScore", "LTeamID", "LScore", "WFGM", "WFGA", "WFGM3", "WFGA3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF", "LFGM", "LFGA", "LFGM3", "LFGA3", "LFTM", "LFTA", "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF"]] = df2.loc[idx,["LTeamID", "LScore", "WTeamID", "WScore", "LFGM", "LFGA", "LFGM3", "LFGA3", "LFTM", "LFTA", "LOR", "LDR", "LAst", "LTO", "LStl", "LBlk", "LPF", "WFGM", "WFGA", "WFGM3", "WFGA3", "WFTM", "WFTA", "WOR", "WDR", "WAst", "WTO", "WStl", "WBlk", "WPF"]].values
df2 = df2.rename(columns={"WTeamID":"HTeamID", "WScore":"HScore", "WFGM":"HFGM", "WFGA":"HFGA", "WFGM3":"HFGM3", "WFGA3":"HFGA3", "WFTM":"HFTM", "WFTA":"HFTA", "WOR":"HOR", "WDR":"HDR", "WAst":"HAst", "WTO":"HTO", "WStl":"HStl", "WBlk":"HBlk", "WPF":"HPF"})
df2

,Season,DayNum,HTeamID,HScore,LTeamID,LScore,WLoc,NumOT,HFGM,HFGA,...,LFTM,LFTA,LOR,LDR,LAst,LTO,LStl,LBlk,LPF,Result
0,2003,10,1328,62,1104,68,N,0,22,53,...,11,18,14,24,13,23,7,1,22,0
1,2003,10,1393,63,1272,70,N,0,24,67,...,10,19,15,28,16,13,4,4,18,0
2,2003,11,1437,61,1266,73,N,0,22,73,...,17,29,17,26,15,10,5,2,25,0
3,2003,11,1457,50,1296,56,N,0,18,49,...,17,31,6,19,11,12,14,2,18,0
4,2003,11,1400,77,1208,71,N,0,30,61,...,17,27,21,15,12,10,7,1,14,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
106829,2023,113,1403,74,1328,63,A,0,29,47,...,12,15,6,13,13,8,10,1,12,1
106830,2023,113,1405,84,1103,63,H,0,32,61,...,7,14,9,23,10,10,3,0,11,1
106831,2023,113,1461,55,1429,65,A,0,17,54,...,16,22,7,27,11,12,7,3,15,0
106832,2023,113,1433,88,1386,63,A,0,31,59,...,13,18,5,20,10,11,3,3,15,1


In [7]:
winners.sort_values('WTeamID')

,Season,DayNum,WTeamID,WScore,WLoc,NumOT,WFGM,WFGA,WFGM3,WFGA3,WFTM,WFTA,WOR,WDR,WAst,WTO,WStl,WBlk,WPF
82785,2019,17,1101,72,N,0,29,56,11,18,3,6,9,20,19,6,4,1,21
101856,2022,128,1101,82,N,0,28,55,11,21,15,19,3,13,15,9,10,1,20
98301,2022,44,1101,69,H,0,17,44,5,17,30,40,9,22,11,20,14,3,22
90020,2020,68,1101,68,H,0,18,52,7,22,25,35,11,20,11,14,14,1,25
63083,2015,71,1101,95,A,0,34,59,16,27,11,12,7,22,17,9,6,0,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105578,2023,87,1477,62,A,0,20,42,10,27,12,15,5,17,11,12,7,4,16
105234,2023,80,1477,63,A,0,25,58,7,24,6,8,4,20,19,9,8,2,16
102705,2023,20,1477,75,N,0,31,66,13,39,0,1,7,23,21,10,7,1,15
103223,2023,30,1477,53,A,0,19,60,4,28,11,15,12,28,9,12,6,2,21


Jason trying to learn some tensorflow

In [8]:
from tensorflow import keras
from tensorflow.keras.datasets import boston_housing
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation

train data: data used for training the model

test data: data used for testing the model

In [9]:
(x_train, y_train), (x_test, y_test) = boston_housing.load_data()


57026/57026 [==============================] - 0s 0us/step


Sequential Model: Straight line of "Tensors", with arbitrary number of "Layers". We use 2 "Dense" layers. Each layer predicts increasingly abstract data. Input dimension specifies size of inputted array. 

Dense Layer: Every neuron connected to a neuron in the previous layer. Output of each neuron is a weighted sum of the inputs plus a "bias" term. 

- First parameter: number of neurons in the layer. Controls complexity and capacity, if too large then overfit risk

- Second parameter: the activation function to use. Determines the output of a neuron given the weighted sum of the inputs plus a bias term. 

- Third parameter: number of input features. Only needed for the first layer, the rest can infer the input_dim based on the previous layer. 

In [10]:
model = Sequential([
    Dense(64, activation="relu", input_dim=13),
    Dense(10, activation="softmax")
])

2023-03-12 22:04:00.811208: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-12 22:04:00.811239: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-12 22:04:00.811253: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (p-76c1b032-5d0a-4d37-bf01-322f3e813bd4): /proc/driver/nvidia/version does not exist
2023-03-12 22:04:00.811450: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


For compile: a million parameters we CAN change if we want. Basically telling the model what we want it to be good at doing. 

Loss is the function that calculates the difference between the actual output and the expected output (trying to minimize this function). Use mean squared error for regression problems, categorical cross-entropy for classification problems. 

Metrics is an array of ways to evaluate the effectiveness of the model based on a training data set. NOT USED DURING TRAINING PROCESS. Use accuracy for classification problems, mean squared error for regression problems. 

Optimizer basically tells model HOW we want it to optimize (depends on nature of problem being solved and size of training data). 

In [11]:
model.compile(
    optimizer=keras.optimizers.RMSprop(),
    loss=keras.losses.MeanSquaredError(),
    metrics=[keras.metrics.MeanSquaredError()]
)

Fit is where we actually train the model with x and y train. Epochs = number of times the model iterates over the training data. 

In [12]:
history = model.fit(
    x_train,
    y_train,
    epochs=2
)

Epoch 1/2
13/13 [==============================] - 0s 2ms/step - loss: 581.7792 - mean_squared_error: 581.7792
Epoch 2/2
13/13 [==============================] - 0s 893us/step - loss: 581.7797 - mean_squared_error: 581.7797


Testing the model on the test data

In [13]:
model.test_on_batch(
    x_test,
    y_test
)

[611.34130859375, 611.34130859375]

Import kenpom stats for every team from years 2002-2017

In [14]:
kenpom_df = pd.read_csv("data/kenpom_2017.csv")

Import tourney game results for every tourney game from years 2002-2017

In [15]:
tgr_df = pd.read_csv("data/MNCAATourneyCompactResults.csv")
tgr_df = tgr_df.loc[tgr_df["Season"] >= 2002]
tgr_df = tgr_df.loc[tgr_df["Season"] <= 2017]

Import kaggle IDs for each team

In [52]:
MTeams = pd.read_csv("data/MTeams.csv")
MTeamSpellings = pd.read_csv("data/MTeamSpellings.csv")
MTeams.drop(columns=["FirstD1Season","LastD1Season"], inplace=True)
cols = MTeams.columns.tolist()
cols = cols[-1:] + cols[:-1]
MTeams = MTeams[cols]
MTeams["TeamName"] = MTeams["TeamName"].str.lower()
MTeamSpellings.rename(columns={"TeamNameSpelling":"TeamName"}, inplace=True)
IDs_df = pd.concat([MTeams, MTeamSpellings])

Change team names in kenpom and tgr to thier kaggle IDs

In [53]:
kenpom_df2 = kenpom_df.copy(deep=True)
kenpom_df2['Team'] = kenpom_df2['Team'].apply(lambda x: x.strip())
kenpom_df2['Team'] = kenpom_df2['Team'].apply(lambda x: x.lower())
kenpom_df2['Team'] = pd.Series(kenpom_df2['Team']).str.replace('\',', '\'', regex=False)
kenpom_df2 = pd.merge(kenpom_df2, IDs_df, left_on="Team", right_on="TeamName", how="left")
# kenpom_df2["Team"] = kenpom_df2["TeamID"]
# kenpom_df2["Team"] = kenpom_df2["Team"].astype(int)
# kenpom_df2["Power5"] = np.where(kenpom_df2["Conference"] == "ACC", 1, 0)
# kenpom_df2["Power5"] = np.where(kenpom_df2["Conference"] == "B10", 1, kenpom_df2["Power5"])
# kenpom_df2["Power5"] = np.where(kenpom_df2["Conference"] == "SEC", 1, kenpom_df2["Power5"])
# kenpom_df2["Power5"] = np.where(kenpom_df2["Conference"] == "B12", 1, kenpom_df2["Power5"])
# kenpom_df2["Power5"] = np.where(kenpom_df2["Conference"] == "P10", 1, kenpom_df2["Power5"])
# kenpom_df2["Power5"] = np.where(kenpom_df2["Conference"] == "P12", 1, kenpom_df2["Power5"])
# kenpom_df2["WPct"] = kenpom_df2["Wins"] / (kenpom_df2["Wins"] + kenpom_df2["Losses"])
# kenpom_df2 = kenpom_df2.drop(columns=["Conference", "Wins", "Losses", "TeamID", "TeamName","AdjustO Rank", "AdjustD Rank","AdjustT Rank","Luck Rank","SOS Pyth Rank","SOS OppO Rank","SOS OppD Rank","NCSOS Pyth Rank"])
kenpom_df2

,Year,Rank,Team,Conference,Wins,Losses,Seed,Pyth,AdjustO,AdjustO Rank,...,SOS Pyth,SOS Pyth Rank,SOS OppO,SOS OppO Rank,SOS OppD,SOS OppD Rank,NCSOS Pyth,NCSOS Pyth Rank,TeamName,TeamID
0,2002,1,duke,ACC,31,4,1.0,34.19,121.0,1,...,9.87,18,109.3,13,99.5,31,6.66,34,duke,1181.0
1,2002,1,duke,ACC,31,4,1.0,34.19,121.0,1,...,9.87,18,109.3,13,99.5,31,6.66,34,duke,1181.0
2,2002,2,cincinnati,CUSA,31,4,1.0,30.19,118.1,7,...,6.58,57,106.5,66,100.0,44,3.48,80,cincinnati,1153.0
3,2002,2,cincinnati,CUSA,31,4,1.0,30.19,118.1,7,...,6.58,57,106.5,66,100.0,44,3.48,80,cincinnati,1153.0
4,2002,3,maryland,ACC,32,4,1.0,29.25,119.2,4,...,9.88,16,109.4,11,99.5,32,1.62,120,maryland,1268.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8828,2017,349,north carolina a&t,MEAC,3,29,NaN,-27.71,95.4,322,...,-9.40,337,98.7,342,108.1,315,0.08,149,north carolina a&t,1299.0
8829,2017,350,presbyterian,BSth,5,25,NaN,-29.20,88.9,349,...,-4.36,261,102.8,250,107.1,270,2.25,83,presbyterian,1342.0
8830,2017,350,presbyterian,BSth,5,25,NaN,-29.20,88.9,349,...,-4.36,261,102.8,250,107.1,270,2.25,83,presbyterian,1342.0
8831,2017,351,alabama a&m,SWAC,2,27,NaN,-30.34,91.4,342,...,-10.15,343,98.7,341,108.9,335,1.08,125,alabama a&m,1105.0


In [54]:
years = kenpom_df2.groupby("Year")
cols = ["Pyth","AdjustO","AdjustD","AdjustT","Luck","SOS Pyth","SOS OppO","SOS OppD","NCSOS Pyth", "WPct"]
i = 0
for year in years:
    year_data = year[1]
    for col in cols:
        year_data[col] = (year_data[col] - year_data[col].min()) / (year_data[col].max() - year_data[col].min())
    if i == 0:
        normal_kenpom_df = year_data.copy(deep=True)
        i += 1
    else:
        normal_kenpom_df = pd.concat([normal_kenpom_df, year_data],ignore_index=True,sort=False)


KeyError: 'WPct'

In [0]:
tgr_df2 = tgr_df.copy(deep= True)
tgr_df2["Result"] = np.select([tgr_df["WTeamID"] > tgr_df["LTeamID"], tgr_df["WTeamID"] < tgr_df["LTeamID"]], [1, 0])
idx = (tgr_df2["Result"] == 0)
tgr_df2.loc[idx,["WTeamID", "WScore", "LTeamID", "LScore"]] = tgr_df2.loc[idx,["LTeamID", "LScore", "WTeamID", "WScore"]].values
tgr_df2 = tgr_df2.rename(columns={"WTeamID":"HTeamID", "WScore":"HScore"})
tgr_df2 = tgr_df2.drop(columns=["DayNum", "NumOT", "HScore", "LScore", "WLoc"])

In [0]:
tgr_w_kenpom_df = pd.merge(tgr_df2, normal_kenpom_df, how='left', left_on=['Season','HTeamID'], right_on=['Year','Team'])
tgr_w_kenpom_df = pd.merge(tgr_w_kenpom_df, normal_kenpom_df, how='left', left_on=['Season','LTeamID'], right_on=['Year','Team'])
tgr_w_kenpom_df = tgr_w_kenpom_df.drop(columns=['Year_x','Year_y', 'Team_x', 'Team_y'])
cols = ["Rank","Seed","Pyth","AdjustO","AdjustD","AdjustT","Luck","SOS Pyth","SOS OppO","SOS OppD","NCSOS Pyth", "Power5","WPct"]

In [0]:
submission_df = pd.DataFrame(list(product(IDs_df['TeamID'], IDs_df['TeamID'])))
submission_df

In [0]:
# 1101 - 1477
temp = [[1102,1101]]
for i in range(1103, 1478):
    for j in range(1101, i):
        temp.append([i,j])
submission_df = pd.DataFrame(temp, columns=["HTeamID","LTeamID"])
lteams = submission_df.LTeamID.values.tolist()
hteams = submission_df.HTeamID.values.tolist()
IDs = []
for i in range(0,70876):
    IDs.append("2023_" + str(lteams[i]) + "_" + str(hteams[i]))
submission_df["ID"] = IDs
submission_df

In [0]:
import requests
from bs4 import BeautifulSoup

In [0]:
def record_to_percent(s):
    n1, n2 = re.findall('(\\d+)-(\\d+)', s)[0]
    n1, n2 = float(n1), float(n2)
    return n1 / (n1 + n2)

In [0]:
bs = BeautifulSoup(open('kenpom2023.html'), 'html.parser')
bs.prettify()
df = pd.read_html(str(bs.find('table')))[0]
kenpom2023_df = df.iloc[:, np.r_[0:6, 7, 9, 11, 13, 15, 17, 19]]
kenpom2023_df.columns = ['Rankk','Team','Conference','W-L','AdjEM','AdjO','AdjD','AdjT','Luck','SOS AdjEM','SOS OppO','SOS OppD','NCSOS']
kenpom2023_df['Team'] = kenpom2023_df['Team'].apply(lambda x: x.strip())
kenpom2023_df['Team'] = pd.Series(kenpom2023_df['Team']).str.replace('.', '', regex=False)
kenpom2023_df['Team'] = pd.Series(kenpom2023_df['Team']).str.replace('\',', '\'', regex=False)
kenpom2023_df['Team'] = kenpom2023_df['Team'].replace({
'Cal St.': 'CS',
'Albany': 'SUNY Albany',
'Abilene Christian': 'Abilene Chr',
'American': 'American Univ',
'Arkansas Little Rock': 'Ark Little Rock',
'Arkansas Pine Bluff': 'Ark Pine Bluff',
'Boston University': 'Boston Univ',
'Central Michigan': 'C Michigan',
'Central Connecticut': 'Central Conn',
'Coastal Carolina': 'Coastal Car',
'East Carolina': 'E Kentucky',
'Eastern Washington': 'E Washington',
'East Tennessee St': 'ETSU',
'Fairleigh Dickinson': 'F Dickinson',
'Florida Atlantic': 'FL Atlantic',
'Florida Gulf Coast': 'FL Gulf Coast',
'George Washington': 'G Washington',
'Illinois Chicago': 'IL Chicago',
'Kent St': 'Kent',
'Monmouth': 'Monmouth NJ',
'Mississippi Valley St': 'MS Valley St',
'Northern Colorado': 'N Colorado',
'North Dakota St': 'N Dakota St',
'Northern Kentucky': 'N Kentucky',
'North Carolina A&T': 'NC A&T',
'North Carolina Central': 'NC Central',
'North Carolina St': 'NC State',
'Northwestern St': 'Northwestern LA',
'Prairie View A&M': 'Prairie View',
'South Carolina St': 'S Carolina St',
'South Dakota St': 'S Dakota St',
'Southern Illinois': 'S Illinois',
'Southeastern Louisiana': 'SE Louisiana',
'Stephen F Austin': 'SF Austin',
'Southern': 'Southern Univ',
'Southern Univ Miss': 'Southern Miss',
'Saint Louis': 'St Louis',
'Texas A&M Corpus Chris': 'TAM C. Christi',
'Troy St': 'Troy',
'Texas Southern Univ': 'TX Southern',
'Louisiana Lafayette': 'Louisiana',
'UTSA': 'UT San Antonio',
'Western Michigan': 'W Michigan',
'Green Bay': 'WI Green Bay',
'Milwaukee': 'WI Milwaukee',
'Western Kentucky': 'WKU',
'College of Charleston': 'Col Charleston',
'Loyola Chicago': 'Loyola-Chicago',
'The Citadel':'Citadel',
'UMKC':"Missouri KC",
'Northern Illinois':'N Illinois',
"Louisiana Monroe":'ULM',
'Middle Tennessee':'MTSU',
'Middle Tennessee St':'MTSU',
'Western Carolina':'W Carolina',
'Eastern Illinois': 'E Illinois',
'FIU':'Florida Intl',
'Tennessee Martin':'TN Martin',
'Western Illinois': 'W Illinois',
'Stephan F Austin':"SF Austin",
'Loyola Marymount':'Loy Marymount',
'Charleston Southern':'Charleston So',
'Bethune Cookman':'Bethune-Cookman',
'Eastern Kentucky':'E Kentucky',
'Maryland Eastern Shore':'MD E Shore',
'Birmingham Southern':'Birmingham So',
'Eastern Michigan':'E Michigan',
'Texas Southern':'TX Southern',
'Georgia Southern': 'Ga Southern',
'Cal St Northridge':'CS Northridge',
'Saint Mary\'s': 'St Mary\'s CA',
'Cal St Fullerton':'CS Fullerton',
'Saint Joseph\'s':'St Joseph\'s PA',
'Southwest Texas St':'Texas St',
'Southeast Missouri St':'SE Missouri St',
'Sacramento St':'CS Sacramento',
'Southwest Missouri St':'Missouri St',
'Texas Pan American':'UTRGV',
'UT Rio Grande Valley':'UTRGV',
'IPFW':'PFW',
'Fort Wayne':'PFW',
'Purdue Fort Wayne':'PFW',
'Saint Peter\'s':'St Peter\'s',
'Grambling St':'Grambling',
'Utah Valley St':'Utah Valley',
'Mount St Mary\'s':'Mt St Mary\'s',
'Kennesaw St':'Kennesaw',
'Central Arkansas':'Cent Arkansas',
'Winston Salem St':'W Salem St',
'Cal St Bakersfield':'CS Bakersfield',
'USC Upstate':'SC Upstate',
'SIU Edwardsville':'SIUE',
'Houston Baptist':'Houston Chr',
'Houston Christian':'Houston Chr',
'Nebraska Omaha':'NE Omaha',
'UMass Lowell':'MA Lowell',
'Little Rock':'Ark Little Rock',
'Charleston':'Col Charleston',
'St Thomas':'St Thomas MN',
'Detroit Mercy':"Detroit",
'Queens':'Queens NC',
'Texas A&M Commerce':'TX A&M Commerce',
'LIU':'LIU Brooklyn'})
kenpom2023_df = pd.merge(kenpom2023_df, IDs_df, left_on="Team", right_on="TeamName", how="left")
kenpom2023_df = kenpom2023_df.dropna()
kenpom2023_df["Team"] = kenpom2023_df["TeamID"]
kenpom2023_df["Team"] = kenpom2023_df["Team"].astype(int)
kenpom2023_df["Power5"] = np.where(kenpom2023_df["Conference"] == "ACC", 1, 0)
kenpom2023_df["Power5"] = np.where(kenpom2023_df["Conference"] == "B10", 1, kenpom2023_df["Power5"])
kenpom2023_df["Power5"] = np.where(kenpom2023_df["Conference"] == "SEC", 1, kenpom2023_df["Power5"])
kenpom2023_df["Power5"] = np.where(kenpom2023_df["Conference"] == "B12", 1, kenpom2023_df["Power5"])
kenpom2023_df["Power5"] = np.where(kenpom2023_df["Conference"] == "P10", 1, kenpom2023_df["Power5"])
kenpom2023_df["Power5"] = np.where(kenpom2023_df["Conference"] == "P12", 1, kenpom2023_df["Power5"])
kenpom2023_df['WPct'] = kenpom2023_df['W-L'].apply(lambda x: record_to_percent(x))
kenpom2023_df = kenpom2023_df.drop(columns=["Conference", "W-L", "TeamID", "TeamName"])

In [0]:
cols = ["AdjEM","AdjO","AdjD","AdjT","Luck","SOS AdjEM","SOS OppO","SOS OppD","NCSOS", "WPct"]
i = 0
for col in cols:
    kenpom2023_df[col] = (kenpom2023_df[col] - kenpom2023_df[col].min()) / (kenpom2023_df[col].max() - kenpom2023_df[col].min())
if i == 0:
    normal_kenpom2023_df = kenpom2023_df.copy(deep=True)
    i += 1
else:
    normal_kenpom2023_df = pd.concat([normal_kenpom2023_df, kenpom2023_df],ignore_index=True,sort=False)

In [0]:
submission_df = pd.merge(submission_df, normal_kenpom2023_df, left_on="HTeamID", right_on="Team", how="left")
submission_df = pd.merge(submission_df, normal_kenpom2023_df, left_on="LTeamID", right_on="Team", how="left")
submission_df.drop(columns=["Team_x", "Team_y"], inplace=True)
submission_df

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=76c1b032-5d0a-4d37-bf01-322f3e813bd4' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>